In [ ]:
# install vertex ai and genai
%pip install --upgrade --quiet google-cloud-aiplatform google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.0 MB/s eta 0:00:00


In [ ]:
import os

from google import genai
import vertexai

PROJECT_ID = "genai-455208"  #[your-project-id] to use vertex ai

In [ ]:
# auth for colab, if local refer https://cloud.google.com/docs/authentication/set-up-adc-local-dev-environment
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [ ]:
from google.genai.types import GenerateContentConfig, Retrieval, Tool, VertexRagStore
from vertexai import rag

In [ ]:
EMBEDDING_MODEL = "publishers/google/models/text-embedding-005"

rag_corpus = rag.create_corpus(
    display_name="Chinese-rag-corpus",
    backend_config=rag.RagVectorDbConfig(
        rag_embedding_model_config=rag.RagEmbeddingModelConfig(
            vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
                publisher_model=EMBEDDING_MODEL
            )
        )
    ),
)

In [ ]:
rag.list_corpora()

ListRagCorporaPager<rag_corpora {
  name: "projects/genai-455208/locations/us-central1/ragCorpora/2305843009213693952"
  display_name: "my-rag-corpus"
  create_time {
    seconds: 1743241187
    nanos: 886217000
  }
  update_time {
    seconds: 1743241187
    nanos: 886217000
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
      knn {
      }
    }
    rag_embedding_model_config {
      vertex_prediction_endpoint {
        endpoint: "projects/genai-455208/locations/us-central1/publishers/google/models/text-embedding-005"
      }
    }
  }
}
rag_corpora {
  name: "projects/genai-455208/locations/us-central1/ragCorpora/6917529027641081856"
  display_name: "my-rag-corpus"
  create_time {
    seconds: 1743241230
    nanos: 37983000
  }
  update_time {
    seconds: 1743241230
    nanos: 37983000
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
      knn {
      }
    }
    rag_embedding_model_config {
      vertex_pr

In [ ]:
%%writefile Chinese_text.md

我们两个月前在公司的种地活动中申请了一块地，取名为"红果与菜"。 我们整理土地后在大爷的指导下种上了豇豆，辣椒和空心菜。
但是经过一个月只长出一颗辣椒，其他的都是野草比如铁苋菜和凹头苋。最近我们把野草都拔掉，种上了蓝莓和胡萝卜。蓝莓是买的树，已经开花了。
胡萝卜也发了小芽。经研究发现野草是旁边地里种子被风刮来的，所以我们合力把旁边地也清理了一番，种上了黄瓜茄子空心菜等，现在也已经发芽了。
勤劳的农民每天都去浇水，当然下雨天除外。期待蔬菜长成和蓝莓果成熟。

Overwriting Chinese_text.md


In [ ]:
rag_file = rag.upload_file(
    corpus_name=rag_corpus.name,
    path="Chinese_text.md",
    display_name="Chinese_text",
    description="description for our field",
)

In [ ]:
# Direct context retrieval
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    rag_retrieval_config=rag.RagRetrievalConfig(
        top_k=5,  # Optional
        filter=rag.Filter(
            vector_distance_threshold=0.5,  # Optional
        ),
    ),
    text="红果与菜种了什么",
)

print(response)

contexts {
  contexts {
    source_uri: "Chinese_text"
    text: "我们两个月前在公司的种地活动中申请了一块地，取名为\"红果与菜\"。 我们整理土地后在大爷的指导下种上了豇豆，辣椒和空心菜。\n但是经过一个月只长出一颗辣椒，其他的都是野草比如铁苋菜和凹头苋。最近我们把野草都拔掉，种上了蓝莓和胡萝卜。蓝莓是买的树，已经开花了。\n胡萝卜也发了小芽。经研究发现野草是旁边地里种子被风刮来的，所以我们合力把旁边地也清理了一番，种上了黄瓜茄子空心菜等，现在也已经发芽了。\n勤劳的农民每天都去浇水，当然下雨天除外。期待蔬菜长成和蓝莓果成熟。"
    source_display_name: "Chinese_text"
    score: 0.45096697957826348
    chunk {
      text: "我们两个月前在公司的种地活动中申请了一块地，取名为\"红果与菜\"。 我们整理土地后在大爷的指导下种上了豇豆，辣椒和空心菜。\n但是经过一个月只长出一颗辣椒，其他的都是野草比如铁苋菜和凹头苋。最近我们把野草都拔掉，种上了蓝莓和胡萝卜。蓝莓是买的树，已经开花了。\n胡萝卜也发了小芽。经研究发现野草是旁边地里种子被风刮来的，所以我们合力把旁边地也清理了一番，种上了黄瓜茄子空心菜等，现在也已经发芽了。\n勤劳的农民每天都去浇水，当然下雨天除外。期待蔬菜长成和蓝莓果成熟。"
    }
  }
}



In [ ]:
# Create a tool for the RAG Corpus
rag_retrieval_tool = Tool(
    retrieval=Retrieval(
        vertex_rag_store=VertexRagStore(
            rag_corpora=[rag_corpus.name],
            similarity_top_k=10,
            vector_distance_threshold=0.7,
        )
    )
)


In [ ]:

MODEL_ID = "gemini-2.0-flash-001"

response = client.models.generate_content(
    model=MODEL_ID,
    contents="红果与菜种了什么",
    config=GenerateContentConfig(tools=[rag_retrieval_tool]),
)

display(response.text)

'“红果与菜”种了豇豆、辣椒、空心菜、蓝莓和胡萝卜。最近他们也在清理过的旁边地上种了黄瓜、茄子和空心菜。\n'